In [1]:
import numpy as np
import pandas as pd
import os
from scipy.interpolate import interp1d
from scipy.stats import pearsonr
from scipy.spatial.distance import euclidean, directed_hausdorff
from fastdtw import fastdtw
import matplotlib.pyplot as plt
import re

In [17]:
# --- Block 1: Diameter Profile Resampling ---
def resample_diameter(diameters, num_points=200):
    """
    Resample diameter values to a fixed number of points using linear interpolation.
    Handles NaN values in the input by interpolating/extrapolating from valid points.
    """
    diameters_arr = np.asarray(diameters).astype(np.float64)

    if diameters_arr.size == 0:
        return np.full(num_points, np.nan)
    
    valid_mask = ~np.isnan(diameters_arr)
    if not np.any(valid_mask): # All NaNs in input
        return np.full(num_points, np.nan)

    x_original_indices = np.arange(len(diameters_arr))
    x_old_valid = x_original_indices[valid_mask]
    diameters_valid = diameters_arr[valid_mask]

    if len(diameters_valid) < 2: # Not enough valid points to interpolate
        fill_val = diameters_valid[0] if len(diameters_valid) == 1 else np.nan
        return np.full(num_points, fill_val)

    # Normalize the positions of the valid points to [0,1] for interpolation
    x_for_interp = np.linspace(0, 1, len(x_old_valid)) # Relative positions of valid points

    f_interp = interp1d(x_for_interp, diameters_valid, kind='linear',
                        bounds_error=False,
                        fill_value=(diameters_valid[0], diameters_valid[-1])) # Extrapolate with edge values

    x_new_normalized = np.linspace(0, 1, num_points)
    return f_interp(x_new_normalized)

# --- Block 2: Comparative Metrics Calculation ---
def compare_diameters(gt, pred, num_points_aorta=200, structure_type="aorta"):
    """
    Compare ground truth and predicted diameters using various metrics.
    Logic is adapted based on structure_type ('aorta' or 'iliac').
    Includes Mean Absolute Percentage Error (MAPE).
    """
    mae, mape, dtw_dist, corr, hausdorff_dist = np.nan, np.nan, np.nan, np.nan, np.nan
    gt_resampled_for_plot, pred_resampled_for_plot = np.array([]), np.array([])
    
    original_gt_len, original_pred_len = 0, 0
    gt_nan_proportion, pred_nan_proportion = np.nan, np.nan
    length_ratio = np.nan

    gt_raw_np = np.asarray(gt).astype(np.float64).flatten()
    pred_raw_np = np.asarray(pred).astype(np.float64).flatten()

    original_gt_len = gt_raw_np.size
    original_pred_len = pred_raw_np.size

    if original_gt_len > 0: gt_nan_proportion = np.isnan(gt_raw_np).mean()
    if original_pred_len > 0: pred_nan_proportion = np.isnan(pred_raw_np).mean()
    
    if original_gt_len > 0 and original_pred_len > 0:
        length_ratio = original_pred_len / float(original_gt_len)
    elif original_gt_len == 0 and original_pred_len == 0:
        length_ratio = 1.0
    
    # Custom DTW Fallback (can be called by both iliac and aorta logic)
    def _custom_dtw(s1, s2):
        n_s1, m_s2 = len(s1), len(s2)
        dtw_matrix = np.full((n_s1 + 1, m_s2 + 1), np.inf)
        dtw_matrix[0, 0] = 0
        for i in range(1, n_s1 + 1):
            for j in range(1, m_s2 + 1):
                cost = abs(s1[i-1] - s2[j-1])
                dtw_matrix[i, j] = cost + min(dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1])
        result_dist = dtw_matrix[n_s1, m_s2]
        return result_dist if not np.isinf(result_dist) else np.nan

    # Function to calculate MAPE robustly
    def _calculate_mape(gt_vals, pred_vals, epsilon=1e-6):
        if gt_vals.size == 0 or pred_vals.size == 0: return np.nan
        
        # Ensure only non-NaN corresponding points are used
        valid_indices = ~np.isnan(gt_vals) & ~np.isnan(pred_vals)
        if not np.any(valid_indices): return np.nan
        
        gt_valid = gt_vals[valid_indices]
        pred_valid = pred_vals[valid_indices]

        # Avoid division by zero or very small numbers in GT
        # Add epsilon to GT to avoid 0 in denominator, or use max()
        # Using max() is often preferred to keep the scaling intuitive
        relative_errors = np.abs((gt_valid - pred_valid) / np.maximum(gt_valid, epsilon))
        
        return np.mean(relative_errors) * 100 # Return as percentage

    try:
        if structure_type == "iliac":
            # print(f"  Processing ILIAC: GT_orig_len={original_gt_len}, Pred_orig_len={original_pred_len}")
            gt_for_mae_corr_mape, pred_for_mae_corr_mape = gt_raw_np.copy(), pred_raw_np.copy()

            if original_gt_len == 0 or original_pred_len == 0:
                if original_gt_len > 0 : gt_resampled_for_plot = gt_raw_np.copy()
                if original_pred_len > 0 : pred_resampled_for_plot = pred_raw_np.copy()
            elif original_gt_len != original_pred_len:
                # print(f"  Iliac: Resampling pred (len {original_pred_len}) to GT len ({original_gt_len}) for MAE/Corr/MAPE.")
                pred_for_mae_corr_mape = resample_diameter(pred_raw_np, num_points=original_gt_len)
            
            gt_resampled_for_plot = gt_for_mae_corr_mape.copy()
            pred_resampled_for_plot = pred_for_mae_corr_mape.copy()

            if gt_for_mae_corr_mape.size > 0 and pred_for_mae_corr_mape.size > 0 and \
               not np.all(np.isnan(gt_for_mae_corr_mape)) and not np.all(np.isnan(pred_for_mae_corr_mape)):
                
                mae = np.nanmean(np.abs(gt_for_mae_corr_mape - pred_for_mae_corr_mape))
                mape = _calculate_mape(gt_for_mae_corr_mape, pred_for_mae_corr_mape) # Calculate MAPE
                
                valid_indices_corr = ~np.isnan(gt_for_mae_corr_mape) & ~np.isnan(pred_for_mae_corr_mape)
                if np.sum(valid_indices_corr) >= 3:
                    corr, _ = pearsonr(gt_for_mae_corr_mape[valid_indices_corr], pred_for_mae_corr_mape[valid_indices_corr])
            
            # DTW (on raw, NaN-filtered profiles)
            try: 
                gt_dtw_input = gt_raw_np[~np.isnan(gt_raw_np)]
                pred_dtw_input = pred_raw_np[~np.isnan(pred_raw_np)]
                if gt_dtw_input.size > 0 and pred_dtw_input.size > 0:
                    dtw_dist, _ = fastdtw(gt_dtw_input.tolist(), pred_dtw_input.tolist(), dist=euclidean)
                    if np.isnan(dtw_dist): raise ValueError("fastdtw returned nan for iliac raw")
                else: dtw_dist = np.nan
            except Exception as dtw_error_iliac:
                # print(f"  DTW error (fastdtw) for iliac raw: {dtw_error_iliac}. Trying custom DTW.")
                if gt_dtw_input.size > 0 and pred_dtw_input.size > 0:
                    dtw_dist = _custom_dtw(gt_dtw_input, pred_dtw_input)
                else: dtw_dist = np.nan
            
            # Hausdorff for Iliac (on raw profiles, normalized positions)
            try: 
                gt_hd_input = gt_raw_np[~np.isnan(gt_raw_np)]
                pred_hd_input = pred_raw_np[~np.isnan(pred_raw_np)]
                if gt_hd_input.size > 0 and pred_hd_input.size > 0:
                    x_gt_norm = np.linspace(0, 1, gt_hd_input.size) if gt_hd_input.size > 1 else np.array([0.5])
                    x_pred_norm = np.linspace(0, 1, pred_hd_input.size) if pred_hd_input.size > 1 else np.array([0.5])
                    
                    gt_points_raw = np.vstack((x_gt_norm, gt_hd_input)).T
                    pred_points_raw = np.vstack((x_pred_norm, pred_hd_input)).T
                    
                    if gt_points_raw.shape[0] > 0 and pred_points_raw.shape[0] > 0:
                        hd_gt_to_pred, _, _ = directed_hausdorff(gt_points_raw, pred_points_raw)
                        hd_pred_to_gt, _, _ = directed_hausdorff(pred_points_raw, gt_points_raw)
                        hausdorff_dist = max(hd_gt_to_pred, hd_pred_to_gt)
            except Exception as hd_error_iliac:
                # print(f"  Hausdorff Distance error for iliac raw: {hd_error_iliac}")
                hausdorff_dist = np.nan

        else: # AORTA PROCESSING
            # print(f"  Processing AORTA: Resampling to {num_points_aorta} pts. GT_orig_len={original_gt_len}, Pred_orig_len={original_pred_len}")
            gt_resampled_aorta = resample_diameter(gt_raw_np, num_points_aorta)
            pred_resampled_aorta = resample_diameter(pred_raw_np, num_points_aorta)

            gt_resampled_for_plot = gt_resampled_aorta.copy()
            pred_resampled_for_plot = pred_resampled_aorta.copy()

            if not np.all(np.isnan(gt_resampled_aorta)) and not np.all(np.isnan(pred_resampled_aorta)):
                mae = np.nanmean(np.abs(gt_resampled_aorta - pred_resampled_aorta))
                mape = _calculate_mape(gt_resampled_aorta, pred_resampled_aorta) # Calculate MAPE

                try: # DTW for Aorta
                    gt_dtw_input = gt_resampled_aorta[~np.isnan(gt_resampled_aorta)]
                    pred_dtw_input = pred_resampled_aorta[~np.isnan(pred_resampled_aorta)]
                    if gt_dtw_input.size > 0 and pred_dtw_input.size > 0:
                        dtw_dist, _ = fastdtw(gt_dtw_input.tolist(), pred_dtw_input.tolist(), dist=euclidean)
                    else: dtw_dist = np.nan
                except Exception as dtw_error_aorta:
                    # print(f"  DTW error (fastdtw) for aorta: {dtw_error_aorta}. Trying custom DTW.")
                    if gt_dtw_input.size > 0 and pred_dtw_input.size > 0:
                         dtw_dist = _custom_dtw(gt_dtw_input, pred_dtw_input)
                    else: dtw_dist = np.nan
                
                try: # Pearson for Aorta
                    valid_indices_corr_aorta = ~np.isnan(gt_resampled_aorta) & ~np.isnan(pred_resampled_aorta)
                    if np.sum(valid_indices_corr_aorta) >= 3:
                        corr, _ = pearsonr(gt_resampled_aorta[valid_indices_corr_aorta], pred_resampled_aorta[valid_indices_corr_aorta])
                except Exception as corr_error_aorta:
                    # print(f"  Correlation error for aorta: {corr_error_aorta}")
                    pass # corr is already np.nan
                
                try: # Hausdorff for Aorta
                    gt_hd_input = gt_resampled_aorta[~np.isnan(gt_resampled_aorta)]
                    pred_hd_input = pred_resampled_aorta[~np.isnan(pred_resampled_aorta)]
                    if gt_hd_input.size > 0 and pred_hd_input.size > 0:
                        gt_points_aorta = np.array([[i, val] for i, val in enumerate(gt_hd_input)])
                        pred_points_aorta = np.array([[i, val] for i, val in enumerate(pred_hd_input)])
                        if gt_points_aorta.shape[0] > 0 and pred_points_aorta.shape[0] > 0:
                            hd_gt_to_pred, _, _ = directed_hausdorff(gt_points_aorta, pred_points_aorta)
                            hd_pred_to_gt, _, _ = directed_hausdorff(pred_points_aorta, gt_points_aorta)
                            hausdorff_dist = max(hd_gt_to_pred, hd_pred_to_gt)
                except Exception as hd_error_aorta:
                    # print(f"  Hausdorff Distance error for aorta: {hd_error_aorta}")
                    hausdorff_dist = np.nan
            # else: print("  Aorta: Empty or all-NaN resampled GT or Pred array, most metrics will be NaN.")

    except Exception as e:
        print(f"‼️ Broad Exception in compare_diameters for structure {structure_type} (GT len {original_gt_len}, Pred len {original_pred_len}): {e}")
        num_return_points = original_gt_len if structure_type == "iliac" and original_gt_len > 0 else (num_points_aorta if structure_type == "aorta" else 0)
        if num_return_points > 0:
            gt_resampled_for_plot = np.full(num_return_points, np.nan)
            pred_resampled_for_plot = np.full(num_return_points, np.nan)

    return {
        "MAE": mae, "MAPE": mape, "DTW": dtw_dist, "Correlation": corr, "Hausdorff": hausdorff_dist,
        "GT_Resampled": gt_resampled_for_plot, "Pred_Resampled": pred_resampled_for_plot,
        "GT_Original_Points": original_gt_len, "Pred_Original_Points": original_pred_len,
        "GT_NaN_Proportion": gt_nan_proportion, "Pred_NaN_Proportion": pred_nan_proportion,
        "Length_Ratio": length_ratio
    }    


# --- Block 3: Profile Visualization ---
# def plot_diameter(gt, pred, case_id, metrics_dict=None, output_dir=None, show=False):
#     """Plot the ground truth and predicted diameter profiles."""
#     gt_plot = np.asarray(gt)
#     pred_plot = np.asarray(pred)

#     if gt_plot.size == 0 and pred_plot.size == 0:
#         print(f"  Plotting: Both GT and Pred arrays empty for {case_id}. Skipping plot generation or making empty plot.")
#         # Optionally create an empty plot with a message if desired
#         # For now, just return if no data.
#         return

#     num_plot_points = gt_plot.size if gt_plot.size > 0 else pred_plot.size
#     if num_plot_points == 0: return # Should be caught above

#     x = np.linspace(0, 1, num_plot_points)
    
#     plt.figure(figsize=(10, 6)) # Slightly wider for metrics text
#     if gt_plot.size > 0:
#         plt.plot(x, gt_plot, label="Ground Truth", marker='o', markersize=3, linestyle='-', alpha=0.8)
    
#     # Ensure pred_plot x-axis matches if its length is different (should be same as gt_plot due to compare_diameters logic)
#     x_pred = np.linspace(0,1, pred_plot.size) if pred_plot.size > 0 else np.array([])
#     if pred_plot.size > 0:
#         plt.plot(x_pred, pred_plot, label="Prediction", marker='x', markersize=3, linestyle='--', alpha=0.8)
    
#     plt.xlabel("Normalized Skeleton Path")
#     plt.ylabel("Diameter (mm)")
#     plt.title(f"Diameter Profile Comparison: {case_id}")
    
#     if metrics_dict:
#         text_items = []
#         for m_key in ["MAE", "DTW", "Correlation", "Hausdorff", "Length_Ratio", "GT_Original_Points", "Pred_Original_Points"]:
#             if m_key in metrics_dict and not (isinstance(metrics_dict[m_key], float) and np.isnan(metrics_dict[m_key])):
#                  val = metrics_dict[m_key]
#                  text_items.append(f"{m_key}: {val:.2f}" if isinstance(val, float) else f"{m_key}: {val}")
#         if text_items:
#             plt.text(0.02, 0.02, "\n".join(text_items), transform=plt.gca().transAxes, 
#                      fontsize=7, verticalalignment='bottom', linespacing=1.5,
#                      bbox=dict(boxstyle='round,pad=0.3', fc='lightyellow', alpha=0.7))

#     if gt_plot.size > 0 or pred_plot.size > 0: plt.legend()
#     plt.grid(True, linestyle=':', alpha=0.7)
#     plt.tight_layout(rect=[0, 0.1, 1, 0.95]) # Adjust layout to make space for text

#     if output_dir:
#         os.makedirs(output_dir, exist_ok=True)
#         plot_path = os.path.join(output_dir, f"{case_id}_diameter_comparison.png")
#         plt.savefig(plot_path)
#         print(f"  Plot saved: {plot_path}")
#     if show: plt.show()
#     plt.close()

# def plot_diameter(gt, pred, case_id, metrics_dict=None, output_dir=None, show=False):
#     """Plot the ground truth and predicted diameter profiles.
#     X-axis: Diameter (mm)
#     Y-axis: Normalized Skeleton Path
#     Metrics box is made larger. Includes MAPE.
#     """
#     gt_plot = np.asarray(gt)
#     pred_plot = np.asarray(pred)

#     if gt_plot.size == 0 and pred_plot.size == 0:
#         plt.figure(figsize=(8, 10)) 
#         plt.text(0.5, 0.5, "No data to plot (GT and Pred empty)", ha='center', va='center', transform=plt.gca().transAxes)
#         plt.title(f"Diameter Profile (No Data): {case_id}")
#         plt.xlabel("Diameter (mm)")
#         plt.ylabel("Normalized Skeleton Path")
#         if output_dir:
#             os.makedirs(output_dir, exist_ok=True)
#             plt.savefig(os.path.join(output_dir, f"{case_id}_diameter_comparison_no_data.png"))
#         if show: plt.show()
#         plt.close()
#         return

#     num_plot_points_gt = gt_plot.size
#     num_plot_points_pred = pred_plot.size
    
#     y_path_gt = np.linspace(0, 1, num_plot_points_gt) if num_plot_points_gt > 0 else np.array([])
#     y_path_pred = np.linspace(0, 1, num_plot_points_pred) if num_plot_points_pred > 0 else np.array([])
    
#     plt.figure(figsize=(8, 10)) 
    
#     if gt_plot.size > 0:
#         plt.plot(gt_plot, y_path_gt, label="Ground Truth", marker='o', markersize=3, linestyle='-', alpha=0.8)
#     if pred_plot.size > 0:
#         plt.plot(pred_plot, y_path_pred, label="Prediction", marker='x', markersize=3, linestyle='--', alpha=0.8)
    
#     plt.ylabel("Normalized Skeleton Path") 
#     plt.xlabel("Diameter (mm)")          
#     plt.title(f"Diameter Profile Comparison: {case_id}")
    
#     if metrics_dict:
#         text_items = []
#         # UPDATED: Added MAPE to display order
#         metrics_to_display = ["MAE", "MAPE", "DTW", "Correlation", "Hausdorff", "Length_Ratio"]
#         for m_key in metrics_to_display:
#             if m_key in metrics_dict and not (isinstance(metrics_dict[m_key], float) and np.isnan(metrics_dict[m_key])):
#                  val = metrics_dict[m_key]
#                  # Format MAPE as a percentage string
#                  if m_key == "MAPE":
#                      text_items.append(f"{m_key}: {val:.2f}%" if pd.notnull(val) else f"{m_key}: NaN")
#                  else:
#                      text_items.append(f"{m_key}: {val:.2f}" if isinstance(val, float) else f"{m_key}: {val}")
#         if text_items:
#             plt.text(0.02, 0.02, "\n".join(text_items), 
#                      transform=plt.gca().transAxes, 
#                      fontsize=9,  
#                      verticalalignment='bottom', 
#                      linespacing=1.5,
#                      bbox=dict(boxstyle='round,pad=0.5',  
#                                fc='lightyellow', 
#                                alpha=0.75))

#     if gt_plot.size > 0 or pred_plot.size > 0:
#         plt.legend(fontsize='large') 
#     plt.grid(True, linestyle=':', alpha=0.7)
#     plt.tight_layout(rect=[0, 0.08, 1, 0.95])

#     if output_dir:
#         os.makedirs(output_dir, exist_ok=True)
#         plot_path = os.path.join(output_dir, f"{case_id}_diameter_comparison_switched_axes.png") 
#         plt.savefig(plot_path)
#     if show:
#         plt.show()
#     plt.close()

def plot_diameter(gt, pred, case_id, metrics_dict=None, output_dir=None, show=False):
    """Plot the ground truth and predicted diameter profiles.
    X-axis: Diameter (mm)
    Y-axis: Normalized Skeleton Path
    Metrics box is made larger and positioned at the bottom-right. Includes MAPE.
    """
    gt_plot = np.asarray(gt)
    pred_plot = np.asarray(pred)

    plt.rcParams.update({
    "font.family": "serif",
    "axes.labelsize": 22,
    "axes.titlesize": 22,
    "xtick.labelsize": 22,
    "ytick.labelsize": 22,
    "legend.fontsize": 22,
    "figure.titlesize": 22,
    "mathtext.fontset": "cm"
    })

    if gt_plot.size == 0 and pred_plot.size == 0:
        plt.figure(figsize=(8, 10))
        plt.text(0.5, 0.5, "No data to plot (GT and Pred empty)", ha='center', va='center', transform=plt.gca().transAxes)
        plt.title(f"Diameter Profile (No Data): {case_id}")
        plt.xlabel("Diameter (mm)")
        plt.ylabel("Normalized Skeleton Path")
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
            plt.savefig(os.path.join(output_dir, f"{case_id}_diameter_comparison_no_data.png"))
        if show: plt.show()
        plt.close()
        return

    num_plot_points_gt = gt_plot.size
    num_plot_points_pred = pred_plot.size

    y_path_gt = np.linspace(0, 1, num_plot_points_gt) if num_plot_points_gt > 0 else np.array([])
    y_path_pred = np.linspace(0, 1, num_plot_points_pred) if num_plot_points_pred > 0 else np.array([])

    plt.figure(figsize=(8, 10))

    if gt_plot.size > 0:
        plt.plot(gt_plot, y_path_gt, label="Ground Truth", marker='o', markersize=3, linestyle='-', alpha=0.8)
    if pred_plot.size > 0:
        plt.plot(pred_plot, y_path_pred, label="Prediction", marker='x', markersize=3, linestyle='--', alpha=0.8)

    plt.ylabel("Normalized Skeleton Path")
    plt.xlabel("Diameter (mm)")
    plt.title(f"Diameter Profile Comparison")

    if metrics_dict:
        text_items = []
        metrics_to_display = ["MAE", "MAPE", "DTW", "Correlation", "Hausdorff", "Length_Ratio"]
        for m_key in metrics_to_display:
            if m_key in metrics_dict and not (isinstance(metrics_dict[m_key], float) and np.isnan(metrics_dict[m_key])):
                 val = metrics_dict[m_key]
                 if m_key == "MAPE":
                     text_items.append(f"{m_key}: {val:.2f}%" if pd.notnull(val) else f"{m_key}: NaN")
                 else:
                     text_items.append(f"{m_key}: {val:.2f}" if isinstance(val, float) else f"{m_key}: {val}")
        if text_items:
            # Position the text box at the bottom-right
            # plt.text(0.98, 0.02, # x, y coordinates (0.98 for right, 0.02 for bottom)
            #          "\n".join(text_items),
            #          transform=plt.gca().transAxes,
            #          fontsize=22,
            #         #  horizontalalignment='right',  # Align text box to the right of the x-coordinate
            #          verticalalignment='bottom',  # Align text box to the bottom of the y-coordinate
            #          horizontalalignment='left', # Align text box to the right of the x-coordinate
            #          linespacing=1.5,
            #          bbox=dict(boxstyle='round,pad=0.5',
            #                    fc='lightyellow',
            #                    alpha=0.75))
            
            plt.text(0.98, 0.02,  # 98% from left (almost right edge), 2% from bottom (almost bottom edge)
            "\n".join(text_items),
            transform=plt.gca().transAxes,
            fontsize=22,
            horizontalalignment='right',     # Align text to the right edge of the textbox
            verticalalignment='bottom',    # Align text to the bottom edge of the textbox
            linespacing=1.5,
            bbox=dict(boxstyle='round,pad=0.5',
                    fc='lightyellow',
                    alpha=0.75))

    if gt_plot.size > 0 or pred_plot.size > 0:
        plt.legend(fontsize='large')
    plt.grid(True, linestyle=':', alpha=0.7)
    # Consider adjusting rect if the new text box position needs more/less margin
    plt.tight_layout(rect=[0, 0.08, 1, 0.95]) # Default: rect=[0, 0, 1, 1]

    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        plot_path = os.path.join(output_dir, f"{case_id}_diameter_comparison_switched_axes.png")
        plt.savefig(plot_path, dpi=400, bbox_inches='tight', transparent=True)  # Save with high DPI and tight bounding box
    if show:
        plt.show()
    plt.close()

# --- Block 4: Batch Evaluation ---
          
def evaluate_all_cases(gt_dir, pred_dir, plot_dir=None, show=False, num_points_aorta=200):
    """Evaluate all cases, adapting logic for aorta vs. iliac."""
    results = []
    if not os.path.isdir(gt_dir):
        print(f"Error: Ground truth directory not found: {gt_dir}")
        return pd.DataFrame(results)
    if not os.path.isdir(pred_dir) and os.path.isdir(gt_dir):
        print(f"Warning: Prediction directory not found: {pred_dir}. Cases from GT will be marked as missing predictions.")

    for filename in sorted(os.listdir(gt_dir)):
        if not filename.endswith(".csv"):
            continue

        case_id_processed = filename.replace(".csv", "")
        print(f"Processing case: {case_id_processed}")
        gt_path = os.path.join(gt_dir, filename)
        pred_path = os.path.join(pred_dir, filename)
        
        current_metrics_dict = {
            # UPDATED: Added MAPE to initialization
            "Case": case_id_processed, "MAE": np.nan, "MAPE": np.nan, "DTW": np.nan, "Correlation": np.nan,
            "Hausdorff": np.nan, "GT_Original_Points": 0, "Pred_Original_Points": 0,
            "GT_NaN_Proportion": np.nan, "Pred_NaN_Proportion": np.nan, "Length_Ratio": np.nan,
            "Status": "Pending"
        }

        if not os.path.exists(pred_path):
            print(f"  Prediction file not found: {pred_path}. Logging as missing.")
            current_metrics_dict["Status"] = "Prediction_File_Missing"
            try: 
                gt_df_temp = pd.read_csv(gt_path)
                if "Diameter" in gt_df_temp.columns:
                    gt_diam_temp = gt_df_temp["Diameter"].dropna()
                    current_metrics_dict["GT_Original_Points"] = gt_diam_temp.size
                    current_metrics_dict["GT_NaN_Proportion"] = gt_df_temp["Diameter"].isnull().mean() if gt_df_temp["Diameter"].size > 0 else np.nan
                    current_metrics_dict["MAPE"] = 100.0
                    current_metrics_dict['Length_Ratio'] = 0.0
                    current_metrics_dict["Correlation"] = 0.0
            except Exception as e_gt_read:
                print(f"  Could not read GT file {gt_path} for missing pred case: {e_gt_read}")
            results.append(current_metrics_dict)
            continue
        
        try:
            gt_df = pd.read_csv(gt_path)
            pred_df = pd.read_csv(pred_path)

            gt_diameters_series = gt_df.get("Diameter")
            pred_diameters_series = pred_df.get("Diameter")

            if gt_diameters_series is None or gt_diameters_series.isnull().all():
                print(f"  'Diameter' column missing or all NaN in GT: {gt_path}")
                current_metrics_dict["Status"] = "GT_Data_Invalid"
                if gt_diameters_series is not None: current_metrics_dict["GT_Original_Points"] = gt_diameters_series.size
                if pred_diameters_series is not None: current_metrics_dict["Pred_Original_Points"] = pred_diameters_series.size
                results.append(current_metrics_dict)
                continue
            if pred_diameters_series is None or pred_diameters_series.isnull().all():
                print(f"  'Diameter' column missing or all NaN in Pred: {pred_path}")
                current_metrics_dict["Status"] = "Pred_Data_Invalid"
                current_metrics_dict["GT_Original_Points"] = gt_diameters_series.size
                current_metrics_dict["MAPE"] = 100.0
                current_metrics_dict['Length_Ratio'] = 0.0
                current_metrics_dict["Correlation"] = 0.0
                if pred_diameters_series is not None: current_metrics_dict["Pred_Original_Points"] = pred_diameters_series.size
                results.append(current_metrics_dict)
                continue
            
            gt_diameters = gt_diameters_series.values
            pred_diameters = pred_diameters_series.values
            
            structure_type = "unknown"
            fn_lower = filename.lower()
            if "aorta" in fn_lower:
                structure_type = "aorta"
            elif any(sub in fn_lower for sub in ["iliac", "_il_", "l_il", "r_il", "left_iliac", "right_iliac", "l_illac", "r_illac"]):
                structure_type = "iliac"
            else:
                print(f"  Warning: Could not robustly determine structure for {filename}, defaulting to 'aorta' logic.")
                structure_type = "aorta" # Fallback

            calculated_metrics = compare_diameters(gt_diameters, pred_diameters, 
                                                   num_points_aorta=num_points_aorta, 
                                                   structure_type=structure_type)
            
            current_metrics_dict.update(calculated_metrics)
            current_metrics_dict["Status"] = "Success"

            if plot_dir and (current_metrics_dict["GT_Resampled"].size > 0 or current_metrics_dict["Pred_Resampled"].size > 0) :
                plot_diameter(
                    current_metrics_dict["GT_Resampled"],
                    current_metrics_dict["Pred_Resampled"],
                    case_id=case_id_processed,
                    metrics_dict=current_metrics_dict,
                    output_dir=plot_dir,
                    show=show
                )
            else:
                if current_metrics_dict["Status"] == "Success":
                     current_metrics_dict["Status"] = "Success_Plot_Skipped_No_Data"
            results.append(current_metrics_dict)
        except Exception as e:
            print(f"‼️ Error processing file {filename}: {e}")
            current_metrics_dict["Status"] = f"Error: {str(e)[:100]}"
            results.append(current_metrics_dict)
            continue
    return pd.DataFrame(results)

# def evaluate_all_cases(gt_dir, pred_dir, plot_dir=None, show=False, num_points_aorta=200, offset=200):
#     """Evaluate diameter prediction accuracy for all cases with numeric-ID-based matching and suffix-preserving filenames."""

#     results = []

#     if not os.path.isdir(gt_dir):
#         print(f"❌ Error: Ground truth directory not found: {gt_dir}")
#         return pd.DataFrame(results)

#     if not os.path.isdir(pred_dir):
#         print(f"⚠️ Warning: Prediction directory not found: {pred_dir}. All cases will be logged as missing.")

#     for filename in sorted(os.listdir(gt_dir)):
#         if not filename.endswith(".csv"):
#             continue

#         # Extract numeric ID prefix from filename
#         match = re.match(r"(\d+)", filename)
#         if not match:
#             print(f"⚠️ Could not extract numeric ID from filename: {filename}")
#             continue

#         case_id_numeric = match.group(1)
#         print(f"🔍 Processing case: {filename} → ID: {case_id_numeric}")
#         gt_path = os.path.join(gt_dir, filename)

#         current_metrics_dict = {
#             "Case": case_id_numeric, "MAE": np.nan, "MAPE": np.nan, "DTW": np.nan, "Correlation": np.nan,
#             "Hausdorff": np.nan, "GT_Original_Points": 0, "Pred_Original_Points": 0,
#             "GT_NaN_Proportion": np.nan, "Pred_NaN_Proportion": np.nan, "Length_Ratio": np.nan,
#             "Status": "Pending"
#         }

#         try:
#             numeric_id = int(case_id_numeric)
#             pred_case_id = numeric_id + offset
#             pred_filename = re.sub(r"^\d+", f"{pred_case_id:03d}", filename)
#             pred_path = os.path.join(pred_dir, pred_filename)
#         except Exception as e:
#             print(f"⚠️ Could not compute prediction filename for {filename}: {e}")
#             current_metrics_dict["Status"] = "Filename_Error"
#             results.append(current_metrics_dict)
#             continue

#         if not os.path.exists(pred_path):
#             print(f"⚠️ Missing prediction file: {pred_filename}")
#             current_metrics_dict["Status"] = "Prediction_File_Missing"
#             try:
#                 gt_df_temp = pd.read_csv(gt_path)
#                 if "Diameter" in gt_df_temp.columns:
#                     gt_diam_temp = gt_df_temp["Diameter"].dropna()
#                     current_metrics_dict["GT_Original_Points"] = gt_diam_temp.size
#                     current_metrics_dict["GT_NaN_Proportion"] = gt_df_temp["Diameter"].isnull().mean()
#                     current_metrics_dict["MAPE"] = 100.0
#                     current_metrics_dict["Length_Ratio"] = 0.0
#                     current_metrics_dict["Correlation"] = 0.0
#             except Exception as e:
#                 print(f"⚠️ Failed to read GT CSV for {filename}: {e}")
#             results.append(current_metrics_dict)
#             continue

#         try:
#             # Load data
#             gt_df = pd.read_csv(gt_path)
#             pred_df = pd.read_csv(pred_path)

#             gt_diam = gt_df.get("Diameter")
#             pred_diam = pred_df.get("Diameter")

#             if gt_diam is None or gt_diam.isnull().all():
#                 print(f"⚠️ GT 'Diameter' missing or all NaN: {filename}")
#                 current_metrics_dict["Status"] = "GT_Data_Invalid"
#                 current_metrics_dict["GT_Original_Points"] = gt_diam.size if gt_diam is not None else 0
#                 current_metrics_dict["Pred_Original_Points"] = pred_diam.size if pred_diam is not None else 0
#                 results.append(current_metrics_dict)
#                 continue

#             if pred_diam is None or pred_diam.isnull().all():
#                 print(f"⚠️ Pred 'Diameter' missing or all NaN: {pred_filename}")
#                 current_metrics_dict["Status"] = "Pred_Data_Invalid"
#                 current_metrics_dict["GT_Original_Points"] = gt_diam.size
#                 current_metrics_dict["MAPE"] = 100.0
#                 current_metrics_dict["Length_Ratio"] = 0.0
#                 current_metrics_dict["Correlation"] = 0.0
#                 current_metrics_dict["Pred_Original_Points"] = pred_diam.size if pred_diam is not None else 0
#                 results.append(current_metrics_dict)
#                 continue

#             gt_values = gt_diam.values
#             pred_values = pred_diam.values

#             # Structure type inference
#             structure_type = "aorta"
#             fn_lower = filename.lower()
#             if any(k in fn_lower for k in ["iliac", "l_il", "r_il", "left_iliac", "right_iliac"]):
#                 structure_type = "iliac"

#             # Metric computation
#             calculated = compare_diameters(
#                 gt_values, pred_values,
#                 num_points_aorta=num_points_aorta,
#                 structure_type=structure_type
#             )
#             current_metrics_dict.update(calculated)
#             current_metrics_dict["Status"] = "Success"

#             # Optional plotting
#             if plot_dir and (
#                 current_metrics_dict.get("GT_Resampled", np.array([])).size > 0 or
#                 current_metrics_dict.get("Pred_Resampled", np.array([])).size > 0
#             ):
#                 plot_diameter(
#                     current_metrics_dict["GT_Resampled"],
#                     current_metrics_dict["Pred_Resampled"],
#                     case_id=case_id_numeric,
#                     metrics_dict=current_metrics_dict,
#                     output_dir=plot_dir,
#                     show=show
#                 )
#             else:
#                 current_metrics_dict["Status"] = "Success_Plot_Skipped_No_Data"

#             results.append(current_metrics_dict)

#         except Exception as e:
#             print(f"‼️ Error processing {filename}: {e}")
#             current_metrics_dict["Status"] = f"Error: {str(e)[:100]}"
#             results.append(current_metrics_dict)
#             continue

#     return pd.DataFrame(results)

def generate_summary_tables(final_results_df, 
                            important_metrics=["MAE", "MAPE", "DTW", "Correlation", "Hausdorff", "Length_Ratio"],
                            output_file_prefix="summary_table"):
    """
    Generates and prints summary tables for each structure type (Configuration_Name)
    showing important metrics and their averages.
    """
    if final_results_df.empty:
        print("Cannot generate summary tables: Input DataFrame is empty.")
        return

    columns_for_summary = ["Case"] + [metric for metric in important_metrics if metric in final_results_df.columns]

    for config_name, group_df in final_results_df.groupby('Configuration_Name'):
        print(f"\n\n--- Summary Table for: {config_name} ---")
        
        display_df = group_df[columns_for_summary].copy()
        
        # --- MODIFICATION START ---
        # Define what constitutes an 'evaluated' case for averaging purposes
        # This includes 'Success' and specific error statuses where metrics are intentionally set
        evaluated_cases_df = group_df[
            group_df['Status'].str.startswith("Success", na=False) |
            (group_df['Status'] == "Prediction_File_Missing") | # Include cases where MAPE was set to 100%
            (group_df['Status'] == "Pred_Data_Invalid")       # Include cases where MAPE was set to 100%
        ].copy() # Ensure we're working with a copy to avoid SettingWithCopyWarning
        
        if not evaluated_cases_df.empty: # Changed from successful_cases_df
            averages = {}
            for metric in important_metrics:
                if metric in evaluated_cases_df.columns: # Changed from successful_cases_df
                    if pd.api.types.is_numeric_dtype(evaluated_cases_df[metric]): # Changed from successful_cases_df
                        mean_val = evaluated_cases_df[metric].mean(skipna=True) # Changed from successful_cases_df
                        averages[metric] = mean_val if not pd.isna(mean_val) else np.nan
                    else:
                        averages[metric] = "N/A (non-numeric)"
                else:
                    averages[metric] = "N/A (missing)"
            # --- MODIFICATION END ---
            
            average_row_data = {"Case": "Average"}
            average_row_data.update(averages)
            average_row_df = pd.DataFrame([average_row_data])
            
            for col in columns_for_summary:
                if col not in average_row_df.columns and col != "Case":
                    average_row_df[col] = np.nan

            summary_table_to_print = pd.concat([display_df, average_row_df[columns_for_summary]], ignore_index=True)
        else:
            print(f"  No evaluated cases to average for {config_name}.") # Updated message
            summary_table_to_print = display_df

        for col in important_metrics:
            if col in summary_table_to_print.columns and pd.api.types.is_numeric_dtype(summary_table_to_print[col]):
                if col == "MAPE":
                    summary_table_to_print[col] = summary_table_to_print[col].map(lambda x: f"{x:.2f}%" if pd.notnull(x) and isinstance(x, (float,int)) else x)
                else:
                    summary_table_to_print[col] = summary_table_to_print[col].map(lambda x: f"{x:.3f}" if pd.notnull(x) and isinstance(x, (float,int)) else x)
        
        print(summary_table_to_print.to_string())
        summary_table_to_print.to_csv(f"{output_file_prefix}_{config_name.replace(' ', '_')}.csv", index=False)
        print(f"  Summary table saved to {output_file_prefix}_{config_name.replace(' ', '_')}.csv")

In [18]:
plt.rcParams.update({
    "font.family": "serif",
    "axes.labelsize": 22,
    "axes.titlesize": 22,
    "xtick.labelsize": 22,
    "ytick.labelsize": 22,
    "legend.fontsize": 22,
    "figure.titlesize": 22,
    "mathtext.fontset": "cm"
})

In [19]:
def run_all_evaluations_orchestrator():
    """
    Orchestrates evaluations for Aorta and Iliac arteries, combines results,
    and generates summary tables.
    """


    all_results_list = []
    default_num_points_for_aorta_resampling = 200 

    configurations = [
        {
            "config_name": "Aorta_Eval", # Unique name for this run configuration
            "gt_dir": "Diameter_Cal_Output_Robust/GT_V3/Aorta",
            "pred_dir": "Diameter_Cal_Output_Robust/Sample_pred/Aorta",
            "plot_dir": "Diameter_Cal_Output_Robust/Sample_pred/Aorta"
        },

        {
            "config_name": "Iliac_Arteries_Eval", # One configuration for all iliacs
            "gt_dir": "Diameter_Cal_Output_Robust/GT_V3/Ilia/",
            "pred_dir": "Diameter_Cal_Output_Robust/Sample_pred/Iliac/",
            "plot_dir": "Diameter_Cal_Output_Robust/Sample_pred/Iliac"
        }
    ]    

    for config in configurations:
        print(f"\n--- Running Evaluation Configuration: {config['config_name']} ---")
        if not os.path.exists(config['gt_dir']) :
            print(f"SKIPPING {config['config_name']}: GT directory not found: {config['gt_dir']}")
            continue
        # Prediction dir existence is checked within evaluate_all_cases per file

        df_structure = evaluate_all_cases(
            gt_dir=config['gt_dir'], pred_dir=config['pred_dir'],
            plot_dir=config['plot_dir'], show=False,
            num_points_aorta=default_num_points_for_aorta_resampling
        )
        
        if not df_structure.empty:
            df_structure['Configuration_Name'] = config['config_name'] 
            all_results_list.append(df_structure)
        else:
            print(f"  No results dataframe generated for configuration {config['config_name']}.")

    final_results_df = pd.DataFrame() # Initialize empty
    if all_results_list:
        final_results_df = pd.concat(all_results_list, ignore_index=True)
        print("\n\n--- Combined Full Evaluation Results (Head) ---")
        print(final_results_df.head().to_string())
        
        output_csv_path = "Diameter_Cal_Output_Robust/Iter4_comparison_v2/combined_diameter_evaluation_results_hybrid.csv"
        final_results_df.to_csv(output_csv_path, index=False)
        print(f"\nFull combined results saved to {output_csv_path}")
        
        # --- Generate and Print Summary Tables ---
        generate_summary_tables(final_results_df)
        
    else:
        print("No results were generated from any configuration to create summary tables.")

    return final_results_df

# if __name__ == '__main__':
#     print("Starting diameter profile evaluation script...")
#     # USER: Make sure to replace "PATH_TO_YOUR_GT/..." etc. in the 
#     # configurations list within run_all_evaluations_orchestrator with your actual directory paths.
    
#     final_summary_df = run_all_evaluations_orchestrator()
    
#     if not final_summary_df.empty:
#         print("\nEvaluation complete.")
#         # Optional: Further global plotting on final_summary_df (e.g., Seaborn boxplots)
#         # ... (Seaborn plotting code as in previous example can be added here) ...
#     else:
#         print("\nEvaluation script ran, but no results were generated.")


In [20]:
# # --- Block 5: Main Orchestration and Combined Analysis ---
# def run_all_evaluations_orchestrator():
#     """
#     Orchestrates evaluations for Aorta and Iliac arteries and combines results.
#     """
#     all_results_list = []
#     # This is the main parameter influencing aorta resampling.
#     # Iliac logic within compare_diameters handles its lengths differently.
#     default_num_points_for_aorta_resampling = 200 

#     # --- Define configurations: UPDATE THESE PATHS ---
#     configurations = [
#         {
#             "config_name": "Aorta_Eval", # Unique name for this run configuration
#             "gt_dir": "Diameter_Cal_Output_Robust/GT/Aorta/Diameters",
#             "pred_dir": "Diameter_Cal_Output_Robust/Pred/iter4_v3/Aorta/Diameters",
#             "plot_dir": "Diameter_Cal_Output_Robust/Iter4_comparison/Aorta"
#         },

#         {
#             "config_name": "Iliac_Arteries_Eval", # One configuration for all iliacs
#             "gt_dir": "Diameter_Cal_Output_Robust/GT/Illiac/Diameters",
#             "pred_dir": "Diameter_Cal_Output_Robust/Pred/iter4_v3/Illiac/Diameters",
#             "plot_dir": "Diameter_Cal_Output_Robust/Iter4_comparison/Iliac"
#         }
#         # Add more configurations if needed (e.g., different datasets, models)
#     ]

#     for config in configurations:
#         print(f"\n--- Running Evaluation Configuration: {config['config_name']} ---")
        
#         df_structure = evaluate_all_cases(
#             gt_dir=config['gt_dir'],
#             pred_dir=config['pred_dir'],
#             plot_dir=config['plot_dir'],
#             show=False, # Set to True for interactive plots, False for batch runs
#             num_points_aorta=default_num_points_for_aorta_resampling
#         )
        
#         if not df_structure.empty:
#             # Add a column to identify which configuration these results belong to
#             df_structure['Configuration_Name'] = config['config_name'] 
#             all_results_list.append(df_structure)
#         else:
#             print(f"  No results generated for configuration {config['config_name']}.")

#     if all_results_list:
#         final_results_df = pd.concat(all_results_list, ignore_index=True)
#         print("\n\n--- Combined Evaluation Results ---")
#         print(final_results_df.to_string())
        
#         output_csv_path = "Diameter_Cal_Output_Robust/Iter4_comparison/combined_diameter_evaluation_results_hybrid.csv"
#         final_results_df.to_csv(output_csv_path, index=False)
#         print(f"\nCombined results saved to {output_csv_path}")
        
#         # Further analysis (e.g., mean MAE per Configuration_Name)
#         if 'MAE' in final_results_df.columns:
#              print("\n--- Mean MAE per Configuration ---")
#              # Filter out rows where MAE might be NaN due to errors or invalid data for that case
#              print(final_results_df.dropna(subset=['MAE']).groupby('Configuration_Name')['MAE'].mean())
#     else:
#         print("No results were generated from any configuration.")

#     return final_results_df if all_results_list else pd.DataFrame()

In [21]:
final_summary_df = run_all_evaluations_orchestrator()


--- Running Evaluation Configuration: Aorta_Eval ---
Processing case: 042_diameters_aorta

--- Running Evaluation Configuration: Iliac_Arteries_Eval ---
SKIPPING Iliac_Arteries_Eval: GT directory not found: Diameter_Cal_Output_Robust/GT_V3/Ilia/


--- Combined Full Evaluation Results (Head) ---
                  Case       MAE      MAPE        DTW  Correlation  Hausdorff  GT_Original_Points  Pred_Original_Points  GT_NaN_Proportion  Pred_NaN_Proportion  Length_Ratio   Status                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [ ]:
# if __name__ == '__main__':
#     # This is the entry point of your script
#     # Make sure to replace "PATH_TO_YOUR_GT/..." etc. in the configurations list above
#     # with your actual directory paths.
    
#     print("Starting diameter profile evaluation script...")
#     final_summary_df = run_all_evaluations_orchestrator()
    
#     if not final_summary_df.empty:
#         print("\nEvaluation complete. Summary DataFrame head:")
#         print(final_summary_df.head())
        
#         # Example: Optional global plotting using Seaborn (requires seaborn installed: pip install seaborn)
#         # try:
#         #     import seaborn as sns
#         #     metrics_to_plot_globally = ["MAE", "DTW", "Correlation", "Hausdorff", "Length_Ratio"]
#         #     for metric in metrics_to_plot_globally:
#         #         if metric in final_summary_df.columns and final_summary_df[metric].notna().any():
#         #             plt.figure(figsize=(8, 6))
#         #             sns.boxplot(x='Configuration_Name', y=metric, data=final_summary_df)
#         #             plt.title(f'Global Distribution of {metric}')
#         #             plt.xticks(rotation=45, ha='right')
#         #             plt.tight_layout()
#         #             global_plot_path = f"global_boxplot_{metric}.png"
#         #             plt.savefig(global_plot_path)
#         #             print(f"Saved global plot: {global_plot_path}")
#         #             # plt.show() # Uncomment to show plots interactively
#         #             plt.close()
#         # except ImportError:
#         #     print("\nSeaborn not installed. Skipping global box plots. To install: pip install seaborn")
#         # except Exception as e_plot:
#         #     print(f"Error during global plotting: {e_plot}")

#     else:
#         print("\nEvaluation script ran, but no results were generated.")